# Capítulo 7 — Não supervisionado + PCA

- KMeans e (opcional) DBSCAN
- PCA para redução de dimensionalidade e visualização


In [ ]:
from sklearn.datasets import load_wine
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

data = load_wine()
X = data.data
X = StandardScaler().fit_transform(X)

pca = PCA(n_components=2)
X_pca = pca.fit_transform(X)

km = KMeans(n_clusters=3, n_init=10, random_state=42)
labels = km.fit_predict(X_pca)

plt.figure()
plt.scatter(X_pca[:,0], X_pca[:,1], c=labels)
plt.title("KMeans sobre PCA (Wine)")
plt.xlabel("PC1"); plt.ylabel("PC2")
plt.show()
